<a href="https://colab.research.google.com/github/enakai00/rl_book_solutions/blob/master/Exercise_5_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 5.12 : Solution

In [0]:
import numpy as np
from numpy import random
import copy

In [0]:
track_img1 = '''
    ##############
   #             G
  #              G
  #              G
 #               G
#                G
#                G
#          #######
#         #
#         #
#         #
#         #
#         #
#         #
 #        #
 #        #
 #        #
 #        #
 #        #
 #        #
 #        #
 #        #
  #       #       
  #       #
  #       #
  #       #
  #       #
  #       #
  #       #
   #      #
   #      #
   #SSSSSS#
'''

In [0]:
track_img2 = '''
                 #################
              ###                G
             #                   G   
            #                    G    
           #                     G    
           #                     G    
           #                     G      
           #                     G       
            #                    G
             #                  ##
              #              ###
              #             #
              #           ##
              #          #
              #         #
             #          #
            #           #
           #            #
          #             #
         #              #
        #               #
       #                #       
      #                 #
     #                  #
    #                   #
   #                    #
  #                     #
 #                      #
#                       #
#                       #
#SSSSSSSSSSSSSSSSSSSSSSS#
'''

In [0]:
def get_track(track_img):
  x_max = max(map(len, track_img.split('\n')))
  track = []
  for line in track_img.split('\n'):
    if line == '':
      continue 
    line += ' ' * x_max
    track.append(list(line)[:x_max])

  return np.array(track)

In [0]:
class Car:
  def __init__(self, track):
    self.path = []
    self.track = track
    self._restart()

  def _restart(self):
    self.vx, self.vy = 0, 0
    self.y = len(self.track) - 1
    while True:
      self.x = random.randint(len(self.track[self.y]))
      if self.track[self.y][self.x] == 'S':
        break

  def get_state(self):
    return self.x, self.y, self.vx, self.vy

  def get_result(self):
    result = np.copy(self.track)
    for (x, y, vx, vy, ax, ay) in self.path:
      result[y][x] = '+'
    return result      

  def _action(self, ax, ay):
    _vx = self.vx + ax
    _vy = self.vy + ay
    if _vx in range(5):
      self.vx = _vx
    if _vy in range(5):
      self.vy = _vy

  def move(self, ax, ay, noise):
    self.path.append((self.x, self.y, self.vx, self.vy, ax, ay))
    if noise and random.random() < 0.1:
      self._action(0, 0)
    else:
      self._action(ax, ay)

    for _ in range(self.vy):
      self.y -= 1
      if self.track[self.y][self.x] == 'G':
        return True
      if self.track[self.y][self.x] == '#':
        self._restart()

    for _ in range(self.vx):
      self.x += 1
      if self.track[self.y][self.x] == 'G':
        return True
      if self.track[self.y][self.x] == '#':
        self._restart()

    return False

In [0]:
def trial(car, policy, epsilon = 0.1, noise=True):
  for _ in range(10000):
    x, y, vx, vy = car.get_state()
    state = "{:02},{:02}:{:02},{:02}".format(x, y, vx, vy)
    if state not in policy.keys():
      policy[state] = (0, 0)

    ax, ay = policy[state]
    if random.random() < epsilon:
      ax, ay = random.randint(-1, 2, 2)
    
    finished = car.move(ax, ay, noise)
    if finished:
      break

In [0]:
def optimal_action(q, x, y, vx, vy):
  optimal = (0, 0)
  q_max = 0
  initial = True
  for ay in range(-1, 2):
    for ax in range(-1, 2):
      sa = "{:02},{:02}:{:02},{:02}:{:02},{:02}".format(x, y, vx, vy, ax, ay)
      if sa not in q.keys():
        q[sa] = -10**10
      if initial or q[sa] > q_max:
        q_max = q[sa]
        optimal = (ax, ay)
        initial = False
  return optimal
  
def run_sampling(track, num=100000):
  policy_t = {}
  policy_b = {}
  q = {}
  c = {}
  epsilon = 0.1

  for i in range(num):
    if i % 2000 == 0:
      print ('.', end='')
    car = Car(track)
    trial(car, policy_b, epsilon, noise=True)
    g = 0
    w = 1
    path = car.path
    path.reverse()
    for x, y, vx, vy, ax, ay in path:
      state = "{:02},{:02}:{:02},{:02}".format(x, y, vx, vy)
      sa = "{:02},{:02}:{:02},{:02}:{:02},{:02}".format(x, y, vx, vy, ax, ay)
      action = (ax, ay)

      g += -1 # Reward = -1 for each step
      if sa not in c.keys():
        c[sa] = 0
      c[sa] += w
      if sa not in q.keys():
        q[sa] = 0
      q[sa] += w*(g-q[sa])/c[sa]
      policy_t[state] = optimal_action(q, x, y, vx, vy)

      if policy_t[state] != action:
        break
      w = w / (1 - epsilon + epsilon/9)
      # b(a|s) = (1 - epsilon) + epsilon / 9
      # 1 - epsilon : chosen with the greedy policy
      # epsilon / 9 : chosen with the random policy
    
    policy_b = copy.copy(policy_t) # Update the behaivor policy

  return policy_t

In [0]:
track = get_track(track_img1)
policy_t = run_sampling(track, 200000)

....................................................................................................

In [0]:
for _ in range(3):
  car = Car(track)
  trial(car, policy_t, 0, noise=False)
  result = car.get_result()
  for line in result:
    print (''.join(line))
  print ()

    ##############
   #             G
  #              G
  #              G
 #            +  G
#                G
#          +     G
#          #######
#        +#       
#         #       
#       + #       
#         #       
#         #       
#       + #       
 #        #       
 #        #       
 #        #       
 #      + #       
 #        #       
 #        #       
 #        #       
 #      + #       
  #       #       
  #       #       
  #       #       
  #     + #       
  #       #       
  #       #       
  #     + #       
   #      #       
   #   +  #       
   #SSS+SS#       

    ##############
   #             G
  #              G
  #              G
 #            +  G
#                G
#          +     G
#          #######
#        +#       
#         #       
#       + #       
#         #       
#         #       
#       + #       
 #        #       
 #        #       
 #        #       
 #      + #       
 #        #       
 #        #       
 #        #

In [0]:
track = get_track(track_img2)
policy_t = run_sampling(track, 200000)

....................................................................................................

In [0]:
for _ in range(3):
  car = Car(track)
  trial(car, policy_t, 0, noise=False)
  result = car.get_result()
  for line in result:
    print (''.join(line))
  print ()

                 #################       
              ###                G       
             #                   G       
            #                    G       
           #                     G       
           #                     G       
           #                     G       
           #                     G       
            #                    G       
             #               +  ##       
              #              ###         
              #             #            
              #          +##             
              #          #               
              #         #                
             #       +  #                
            #           #                
           #            #                
          #             #                
         #       +      #                
        #               #                
       #                #                
      #                 #                
     #       +          #         